In [ ]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_excel('sample.xlsx', sheet_name='一级考核-错号率')
raw = df.copy(deep=True)
raw.head()

In [ ]:
def simplify(raw: pd.DataFrame):    
    labels = ['行标签', '总计', '接触量', '错号数', '错号量']
    for name in raw.columns.tolist():
        if name not in labels:
            raw.drop(name, axis=1, inplace=True) 
simplify(raw)
raw.head(10)

In [ ]:
raw.to_excel('sampleTest.xlsx', index=False)

In [ ]:
city_names = []
deprecated_names = ['集团', '上海区域']
for name in raw['行标签'].tolist():
    if name not in deprecated_names and '准业主' not in name \
                                    and '磨合期' not in name \
                                    and '老业主' not in name \
                                    and '稳定期' not in name:
        city_names.append(name)

print(city_names)
len(city_names)

In [ ]:
row_records = []
for _, row in raw.iterrows():
    row_records.append([row['行标签'], row['总计'], row['错号数']])
    # print(row['行标签'], row['总计'], row['错号数'])
row_records

In [ ]:
wrong2city = {}
city_appeared = []
cities = []
slice_idx = []
zyz = []
mhq = []
wdq = []
lyz = []

for i in range(2, len(row_records)):
    tag = row_records[i][0]
    if tag in city_names:
        wrong2city[row_records[i][2]] = tag
        city_appeared.append(tag)
        slice_idx.append(i)
slice_idx.append(len(row_records))
for i in range(len(slice_idx)-1):
    cities.append(row_records[slice_idx[i]:slice_idx[i+1]])
print(cities[0])    
print(city_appeared)
print(slice_idx)

In [ ]:
for city_info in cities:
    CITY = city_info[0][0]
    for line in city_info[1:]:
        if '准业主' in line[0]:
            line[0] = CITY + '-' + line[0]
            zyz.append(line)
        if '磨合期' in line[0]:
            line[0] = CITY + '-' + line[0]
            mhq.append(line)
        if '稳定期' in line[0]:
            line[0] = CITY + '-' + line[0]
            wdq.append(line)
        if '老业主' in line[0]:
            line[0] = CITY + '-' + line[0]
            lyz.append(line)
zyz

In [ ]:
def analyze(l: list):
    res = {}
    for i in range(len(l)):
        city = l[i][0].split('-')[0]
        if not res.__contains__(city):
            res[city] = [l[i][1], l[i][2]]
        else:
            res[city][0] += l[i][1]
            res[city][1] += l[i][2]
    return res
print(analyze(zyz))
print(analyze(mhq))
print(analyze(wdq))
print(analyze(lyz))

In [ ]:
def calculate(d: dict):
    for k in d.keys():
        print(k, d[k])
        d[k].append(float('{:.5f}'.format(d[k][1]/d[k][0])))
    return d
print(calculate(analyze(zyz)))
print(calculate(analyze(mhq)))
print(calculate(analyze(wdq)))
print(calculate(analyze(lyz)))

In [ ]:
zyz_list = []
mhq_list = []
wdq_list = []
lyz_list = []
for k, v in calculate(analyze(zyz)).items():
    zyz_list.append(tuple([k, '准业主'] + v))
for k, v in calculate(analyze(mhq)).items():
    mhq_list.append(tuple([k, '磨合期'] + v))
for k, v in calculate(analyze(wdq)).items():
    wdq_list.append(tuple([k, '稳定期'] + v))
for k, v in calculate(analyze(lyz)).items():
    lyz_list.append(tuple([k, '老业主'] + v))

In [ ]:
labels = ['城市', '业主阶段', '接触量', '错号数', '错号率']
zyz_xlxs = pd.DataFrame(data=zyz_list, columns=labels)
zyz_xlxs = zyz_xlxs.sort_values(by='错号率', ascending=False)
zyz_xlxs.head(20)

In [ ]:
mhq_xlxs = pd.DataFrame(data=mhq_list, columns=labels)
mhq_xlxs = mhq_xlxs.sort_values(by='错号率', ascending=False)
mhq_xlxs.head(20)

In [ ]:
wdq_xlxs = pd.DataFrame(data=wdq_list, columns=labels)
wdq_xlxs = wdq_xlxs.sort_values(by='错号率', ascending=False)
wdq_xlxs.head(20)

In [ ]:
lyz_xlxs = pd.DataFrame(data=lyz_list, columns=labels)
lyz_xlxs = lyz_xlxs.sort_values(by='错号率', ascending=False)
lyz_xlxs.head(20)

In [ ]:
# Shanghai
df = pd.read_excel('sample.xlsx', sheet_name='上海区域')
raw = df.copy(deep=True)
raw.head()
simplify(raw)
raw.head(5)

In [ ]:
for _, rows in raw.iterrows():
    if '准业主' in rows['行标签']:
        # tuple = ('上海区域', '准业主', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量'])))
        # print(('上海区域', '准业主', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量']))))
        shzyz = pd.DataFrame({'城市': ['上海区域'],
                             '业主阶段': ['准业主'],
                             '接触量': rows['接触量'],
                             '错号数': rows['错号量'],
                             '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
        zyz_xlxs = pd.concat([zyz_xlxs, shzyz])

    if '磨合期' in rows['行标签']:
        print(('上海区域', 'mhq', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量']))))
        shmhq = pd.DataFrame({'城市': ['上海区域'],
                             '业主阶段': ['磨合期'],
                             '接触量': rows['接触量'],
                             '错号数': rows['错号量'],
                             '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
        
        mhq_xlxs = pd.concat([mhq_xlxs, shmhq])
    if '稳定期' in rows['行标签']:
        print(('上海区域', 'wdq', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量']))))
        shwdq = pd.DataFrame({'城市': ['上海区域'],
                             '业主阶段': ['稳定期'],
                             '接触量': rows['接触量'],
                             '错号数': rows['错号量'],
                             '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
        wdq_xlxs = pd.concat([wdq_xlxs, shwdq])
    if '老业主' in rows['行标签']:
        print(('上海区域', 'lyz', rows['接触量'], rows['错号量'], float('{:.5f}'.format(rows['错号量']/rows['接触量']))))
        shlyz = pd.DataFrame({'城市': ['上海区域'],
                             '业主阶段': ['老业主'],
                             '接触量': rows['接触量'],
                             '错号数': rows['错号量'],
                             '错号率': float('{:.5f}'.format(rows['错号量']/rows['接触量']))})
        lyz_xlxs = pd.concat([lyz_xlxs, shlyz])

zyz_xlxs.head(30)

In [ ]:
mhq_xlxs.head(20)

In [ ]:
wdq_xlxs.head(10)

In [ ]:
lyz_xlxs.head(10)

In [ ]:
# funtion
def multiple_dfs(df_list, sheets, file_name, spaces):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    row = 0
    for dataframe in df_list:
        dataframe.to_excel(writer,sheet_name=sheets,startrow=row , startcol=0, index=False)   
        row = row + len(dataframe.index) + spaces + 1
    writer.save()

# list of dataframes
dfs = [zyz_xlxs, mhq_xlxs, wdq_xlxs, lyz_xlxs]

# run function
multiple_dfs(dfs, 'Validation', 'test1.xlsx', 1)